## 1.复现上述使用高级API实现的模型

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 下载MNIST数据集并生成DataSet对象
# 使用OneHot编码处理标记
mnist = input_data.read_data_sets("/home/liuzixuan/机器学习/深度/小练习/MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
with tf.Graph().as_default() as g:
    # 输入、标记占位符
    inputs = tf.placeholder(shape=[None, 784], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    
    # 创建128个隐藏层神经元参数
    hidden_weight = tf.Variable(tf.random_normal([784, 128]), name='hidden_weight')
    hidden_bias = tf.Variable(tf.zeros([128, ]), name='hidden_bias')
    # 隐藏层前向传播
    hidden_output = tf.nn.relu(tf.matmul(inputs, hidden_weight) + hidden_bias)
    
    
    # 创建输出层10个神经元参数
    output_weight = tf.Variable(tf.random_normal([128, 10], name='output_weight'))
    output_bias = tf.Variable(tf.zeros([10, ]), name='output_bias')
    # 输出层前向传播
    logits = tf.matmul(hidden_output, output_weight) + output_bias
    output = tf.nn.softmax(logits)
    
    
    # 代价函数
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=labels))
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 114.3885, acc 0.1179
step   500, loss 5.1310, acc 0.7533
step  1000, loss 4.0867, acc 0.8137
step  1500, loss 3.3145, acc 0.8360
step  2000, loss 0.0475, acc 0.8466
step  2500, loss 2.2935, acc 0.8569
step  3000, loss 3.2364, acc 0.8637
step  3500, loss 2.2068, acc 0.8654
step  4000, loss 2.5488, acc 0.8736
step  4500, loss 1.2280, acc 0.8776
step  5000, loss 0.3639, acc 0.8803
step  5500, loss 1.6963, acc 0.8831
step  6000, loss 0.7630, acc 0.8834
step  6500, loss 1.2554, acc 0.8839
step  7000, loss 2.3994, acc 0.8856
step  7500, loss 0.3354, acc 0.8853
step  8000, loss 1.1181, acc 0.8884
step  8500, loss 0.8470, acc 0.8893
step  9000, loss 0.4253, acc 0.8909
step  9500, loss 1.4272, acc 0.8918
step 10000, loss 0.4894, acc 0.8921
step 10500, loss 1.4548, acc 0.8931
step 11000, loss 2.1723, acc 0.8933
step 11500, loss 0.0012, acc 0.8957
step 12000, loss 0.1002, acc 0.8950
step 12500, loss 0.9123, acc 0.8949
step 13000, loss 1.1525, acc 0.8963
step 13500, loss 0.5771, a

In [6]:
with tf.Graph().as_default() as g:
    # 输入、标记占位符
    inputs = tf.placeholder(shape=[None, 784], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    # 隐藏层与输出层
    hidden_output = tf.keras.layers.Dense(
        128, activation=tf.nn.relu)(inputs)
    
    logits = tf.keras.layers.Dense(
        10, activation=None)(hidden_output)
    output = tf.nn.softmax(logits)
    
    # 代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=labels))
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 2.3587, acc 0.0898
step   500, loss 0.6398, acc 0.8546
step  1000, loss 0.3573, acc 0.8879
step  1500, loss 0.5553, acc 0.8991
step  2000, loss 0.3808, acc 0.9062
step  2500, loss 0.3386, acc 0.9083
step  3000, loss 0.4919, acc 0.9132
step  3500, loss 0.6852, acc 0.9178
step  4000, loss 0.2629, acc 0.9204
step  4500, loss 0.3801, acc 0.9204
step  5000, loss 0.1363, acc 0.9249
step  5500, loss 0.0648, acc 0.9241
step  6000, loss 0.0802, acc 0.9266
step  6500, loss 0.5294, acc 0.9287
step  7000, loss 0.6617, acc 0.9325
step  7500, loss 0.3120, acc 0.9325
step  8000, loss 0.3888, acc 0.9325
step  8500, loss 0.1861, acc 0.9351
step  9000, loss 0.4032, acc 0.9342
step  9500, loss 0.1506, acc 0.9374
step 10000, loss 0.0687, acc 0.9366
step 10500, loss 0.1012, acc 0.9370
step 11000, loss 0.1640, acc 0.9391
step 11500, loss 0.1264, acc 0.9399
step 12000, loss 0.1681, acc 0.9439
step 12500, loss 0.2316, acc 0.9421
step 13000, loss 0.1549, acc 0.9422
step 13500, loss 0.1760, acc

## 2.比较使用高级API与使用低级API模型性能的差别，分析可能造成的原因。

#### 高级API是已经封装好的，使用起来简单，但对于某些特殊需求来说，不能细化控制，且一般情况下，使用高级API的模型性能比使用低级API的模型性能要好。
#### 因为高级API中有许多参数已经被充分优化，许多细节已经被考虑到了，不会出现像之前在mnist分类中计算loss时为防止溢出（保证大于0）和过拟合时要加一个极小值这样的操作，加入极小值虽防止溢出和过拟合了，但是却相当于加入了噪声，从而导致对模型性能造成一定的影响，然而使用高级API却不会出现这样的情况。